In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import time
import json
import requests
import matplotlib.pyplot as plt


In [2]:
csv = "oildataRAW.csv"
oil_df = pd.read_csv(csv)
oil_df = oil_df.fillna(0)
oil_df.head()


,Report Number,Supplemental Number,Accident Year,Accident Date/Time,Operator ID,Operator Name,Pipeline/Facility Name,Pipeline Location,Pipeline Type,Liquid Type,...,Other Fatalities,Public Fatalities,All Fatalities,Property Damage Costs,Lost Commodity Costs,Public/Private Property Damage Costs,Emergency Response Costs,Environmental Remediation Costs,Other Costs,All Costs
0,20100016,17305,2010,1/1/2010 7:15,32109,ONEOK NGL PIPELINE LP,KINDER MORGAN JCT,ONSHORE,ABOVEGROUND,"HVL OR OTHER FLAMMABLE OR TOXIC FLUID, GAS",...,0.0,0.0,0.0,110.0,1517.0,0.0,0.0,0.0,0.0,1627
1,20100254,17331,2010,1/4/2010 8:30,15786,PORTLAND PIPELINE CORP,24-INCH MAIN LINE,ONSHORE,ABOVEGROUND,CRUDE OIL,...,0.0,0.0,0.0,4000.0,8.0,0.0,0.0,0.0,0.0,4008
2,20100038,17747,2010,1/5/2010 10:30,20160,"PETROLOGISTICS OLEFINS, LLC",0,ONSHORE,ABOVEGROUND,"HVL OR OTHER FLAMMABLE OR TOXIC FLUID, GAS",...,0.0,0.0,0.0,0.0,200.0,0.0,0.0,0.0,0.0,200
3,20100260,18574,2010,1/6/2010 19:30,11169,"ENBRIDGE ENERGY, LIMITED PARTNERSHIP",SUPERIOR TERMINAL,ONSHORE,UNDERGROUND,CRUDE OIL,...,0.0,0.0,0.0,200.0,40.0,0.0,11300.0,0.0,0.0,11540
4,20100030,16276,2010,1/7/2010 13:00,300,"PLAINS PIPELINE, L.P.",RED RIVER EAST,ONSHORE,UNDERGROUND,CRUDE OIL,...,0.0,0.0,0.0,20000.0,150.0,0.0,7500.0,2000.0,0.0,29650


In [3]:
oil_df['Accident State'].factorize()

(array([ 0,  1,  2, ...,  6, 21,  4]),
 Index(['KS', 'ME', 'LA', 'WI', 'TX', 'ND', 'OK', 'IL', 'MN', 'NY', 'CA', 'IN',
        'CO', 'MS', 'NJ', 'WA', 'IA', 'NC',    0, 'MO', 'NM', 'PA', 'FL', 'VA',
        'WY', 'KY', 'TN', 'MI', 'ID', 'GA', 'NV', 'OH', 'SD', 'AK', 'SC', 'UT',
        'NE', 'MT', 'AL', 'AR', 'MD', 'PR', 'CT', 'OR', 'WV', 'HI', 'MA'],
       dtype='object'))

In [4]:
states = ['KS', 'ME', 'LA', 'WI', 'TX', 'ND', 'OK', 'IL', 'MN', 'NY', 'CA', 'IN',
        'CO', 'MS', 'NJ', 'WA', 'IA', 'NC', 'MO', 'NM', 'PA', 'FL', 'VA', 'WY',
        'KY', 'TN', 'MI', 'ID', 'GA', 'NV', 'OH', 'SD', 'AK', 'SC', 'UT', 'NE',
        'MT', 'AL', 'AR', 'MD', 'PR', 'CT', 'OR', 'WV', 'HI', 'MA']
len(states)

46

In [5]:
states_2 = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
non_accident_states = list(set(states_2) - set(states))
non_accident_states

['AZ', 'VT', 'RI', 'NH', 'DE', 'DC']

In [7]:
oil_df['All Fatalities'].sum()


10.0

In [8]:
oil_df['All Fatalities'].max()


2.0

In [9]:
oil_df.loc[oil_df['All Fatalities'] == 2]

,Report Number,Supplemental Number,Accident Year,Accident Date/Time,Operator ID,Operator Name,Pipeline/Facility Name,Pipeline Location,Pipeline Type,Liquid Type,...,Other Fatalities,Public Fatalities,All Fatalities,Property Damage Costs,Lost Commodity Costs,Public/Private Property Damage Costs,Emergency Response Costs,Environmental Remediation Costs,Other Costs,All Costs
758,20120098,17932,2012,3/3/2012 2:04,11169,"ENBRIDGE ENERGY, LIMITED PARTNERSHIP",LINE 64,ONSHORE,ABOVEGROUND,CRUDE OIL,...,0.0,2.0,2.0,864819.0,124500.0,0.0,915512.0,931946.0,0.0,2836777
2730,20160391,21890,2016,10/31/2016 14:51,2552,COLONIAL PIPELINE CO,LINE 01,ONSHORE,UNDERGROUND,"REFINED AND/OR PETROLEUM PRODUCT (NON-HVL), LI...",...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [10]:
worst_fatalty_days = oil_df.loc[oil_df['All Fatalities'] == 2]['Accident Date/Time']
oil_df.loc[oil_df['All Fatalities'] == 2]['Accident Date/Time']


758        3/3/2012 2:04
2730    10/31/2016 14:51
Name: Accident Date/Time, dtype: object

In [11]:
oil_df['All Injuries'].sum()


20.0

In [12]:
oil_df['All Injuries'].max()


4.0

In [13]:
oil_df.loc[oil_df['All Injuries'] == 4]


,Report Number,Supplemental Number,Accident Year,Accident Date/Time,Operator ID,Operator Name,Pipeline/Facility Name,Pipeline Location,Pipeline Type,Liquid Type,...,Other Fatalities,Public Fatalities,All Fatalities,Property Damage Costs,Lost Commodity Costs,Public/Private Property Damage Costs,Emergency Response Costs,Environmental Remediation Costs,Other Costs,All Costs
2730,20160391,21890,2016,10/31/2016 14:51,2552,COLONIAL PIPELINE CO,LINE 01,ONSHORE,UNDERGROUND,"REFINED AND/OR PETROLEUM PRODUCT (NON-HVL), LI...",...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [14]:
worst_injury_days = oil_df.loc[oil_df['All Injuries'] == 2]['Accident Date/Time']
oil_df.loc[oil_df['All Injuries'] == 2]['Accident Date/Time']

1385    9/30/2013 18:45
Name: Accident Date/Time, dtype: object